In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
df_1 = pd.read_csv("/kaggle/input/titanic/train.csv")
df_2 = pd.read_csv("/kaggle/input/titanic/test.csv")

missing = df_1["Embarked"].isnull()
df_1.drop(missing[missing].index, inplace=True)
df1_cat = df_1[["Sex","Pclass", "SibSp", "Parch", "Embarked"]]
df2_cat = df_2[["Sex","Pclass", "SibSp", "Parch", "Embarked"]]

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

one_hot_vars_1 = pd.get_dummies(df1_cat)
one_hot_vars_2 = pd.get_dummies(df2_cat)

scaler = StandardScaler()
scaler.fit(df_1["Fare"].values.reshape(-1,1))
df_1["Fare"] = scaler.transform(df_1["Fare"].values.reshape(-1,1))

x_train = pd.concat([one_hot_vars_1, df_1["Fare"]], axis=1)
y_train = df_1["Survived"]

mean_imputer = SimpleImputer(strategy='mean')
test_cont = df_2[['Fare']]
test_cont['Fare'] = mean_imputer.fit_transform(test_cont['Fare'].values.reshape(-1,1))
scaler.fit(test_cont['Fare'].values.reshape(-1,1))
test_cont['Fare'] = scaler.transform(test_cont['Fare'].values.reshape(-1,1))
x_test = pd.concat([one_hot_vars_2, test_cont["Fare"]], axis=1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [4]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)
data = {'PassengerId':df_2["PassengerId"], 'Survived': y_pred}
df_final = pd.DataFrame(data)
df_final.to_csv("submission.csv", index=False)